In [ ]:
import matplotlib.pyplot as plt

In [ ]:
with open('output/two_bible_lines.log', 'r') as f:
    lines = f.readlines()

In [ ]:
lines = [l.strip() for l in lines if 'LOG' in l]

In [ ]:
epoch_line_indices = [i for i, line in enumerate(lines) if 'epoch' in line]

In [ ]:
epoch_lines = {epoch: lines[epoch_line_index+1:(epoch_line_indices[epoch+1] if epoch < len(epoch_line_indices) - 1 else len(lines))] for epoch, epoch_line_index in enumerate(epoch_line_indices)}

In [ ]:
epoch_batch_lines = {}
for epoch, el in epoch_lines.items():
    batch_line_indices = [i for i, line in enumerate(el) if 'batch' in line]
    batch_lines = {batch: el[batch_line_index+1:(batch_line_indices[batch+1] if batch < len(batch_line_indices) - 1 else len(el))] for batch, batch_line_index in enumerate(batch_line_indices)}
    epoch_batch_lines[epoch] = batch_lines

In [ ]:
epoch_batch_layer_lines = {}
for epoch, batch_lines in epoch_batch_lines.items():
    epoch_batch_layer_lines[epoch] = {}
    for batch, bl in batch_lines.items():
        layer_line_indices = [i for i, line in enumerate(bl) if 'layer' in line]
        layer_lines = {bl[layer_line_indices[layer]]: bl[layer_line_index+1:(layer_line_indices[layer+1] if layer < len(layer_line_indices) - 1 else len(bl))] for layer, layer_line_index in enumerate(layer_line_indices)}
        epoch_batch_layer_lines[epoch][batch] = layer_lines

In [ ]:
for i, epoch in epoch_batch_layer_lines.items():
    for j, batch in epoch.items():
        for k, layer in batch.items():
            parameter_values = {}
            for l, p in enumerate(layer):
                values = []
                for v in p[16:].split():
                    if 'array' in v:
                        continue
                    numv = v.replace(',', '').replace('[', '').replace(']', '')
                    if not numv.strip():
                        continue
                    values.append(float(numv))
                parameter_values[l] = values
            epoch_batch_layer_lines[i][j][k] = parameter_values

Now we can traverse this dictionary of dictionaries of dictionaries looking for large values. Or we can make a histogram of all the values

In [ ]:
max_value = 0
for i, epoch in epoch_batch_layer_lines.items():
    for j, batch in epoch.items():
        for k, layer in batch.items():
            for l, p in layer.items():
                for el in p:
                    if el > max_value:
                        max_value = el

In [ ]:
print(max_value)

In [ ]:
all_values = []
for i, epoch in epoch_batch_layer_lines.items():
    for j, batch in epoch.items():
        for k, layer in batch.items():
            for l, p in layer.items():
                for el in p:
                    all_values.append(el)

In [ ]:
plt.hist(all_values)
plt.yscale('log')
plt.show()

And now we can use this to detect very large values